In [1]:
from diffusers import DiffusionPipeline
import os
import torch
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(__file__))))))
os.chdir('/nfs/homedirs/rachwan/Torch-Pruning')
import torch_pruning as tp
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
print(pipeline.vae)
# 1. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(pipeline.vae, example_inputs=torch.randn(1,3,224,224))
# pipeline.to("cuda")
# pipeline("An image of a squirrel in Picasso style").images[0]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/nfs/homedirs/rachwan/miniconda3/envs/structure_pruning/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
          (1): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3,

In [2]:
# print([name for name, _ in pipeline.vae.named_modules()])
# 2. Specify the to-be-pruned channels. Here we prune those channels indexed by [2, 6, 9].
pruning_idxs = [2, 6, 9]
pruning_group = DG.get_pruning_group(pipeline.vae.encoder.conv_in, tp.prune_conv_out_channels, idxs=pruning_idxs )

print(pruning_group)

# 3. prune all grouped layers that are coupled with model.conv1 (included).
if DG.check_pruning_group(pruning_group): # avoid full pruning, i.e., channels=0.
    pruning_group.prune()


--------------------------------
          Pruning Group
--------------------------------
[0] [DEP] prune_out_channels on encoder.conv_in (Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))) => prune_out_channels on encoder.conv_in (Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), #Pruned=3
[1] [DEP] prune_out_channels on encoder.conv_in (Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))) => prune_out_channels on _ElementWiseOp(AddBackward0), #Pruned=3
[2] [DEP] prune_out_channels on encoder.conv_in (Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))) => prune_out_channels on _ElementWiseOp(NativeGroupNormBackward0), #Pruned=3
[3] [DEP] prune_out_channels on _ElementWiseOp(NativeGroupNormBackward0) => prune_out_channels on _ElementWiseOp(SiluBackward0), #Pruned=3
[4] [DEP] prune_out_channels on _ElementWiseOp(SiluBackward0) => prune_in_channels on encoder.down_blocks.0.resnets.0.conv1 (Conv2d(128, 128, kernel_size=(3, 3)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7                                                                                             │
│    8 # 3. prune all grouped layers that are coupled with model.conv1 (included).                 │
│    9 if DG.check_pruning_group(pruning_group): # avoid full pruning, i.e., channels=0.           │
│ ❱ 10 │   pruning_group.prune()                                                                   │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DependencyGroup' object has no attribute 'prune'